# Qwen function call

## 直接使用模型

In [6]:
%%time

import openai
import json
import time

openai.api_base = 'http://192.168.0.50:8000/v1'
openai.api_key = 'none'


def call_qwen(messages):
    response = openai.ChatCompletion.create(model='Qwen', messages=messages)
    response = response.choices[0]['message']
    response = json.loads(json.dumps(response, ensure_ascii=False))
    
    print('input:', messages)
    print('output:', response)
    
    return response


if __name__ == '__main__':
    start_time = time.time()
    messages = [{'role': 'user', 'content': '你好'}]
    response = call_qwen(messages)

    execution_time = time.time() - start_time
    print(f"Execution time: {execution_time:.6f} seconds")

input: [{'role': 'user', 'content': '你好'}]
output: {'role': 'assistant', 'content': '你好！很高兴为你服务。有什么我可以帮助你的吗？', 'function_call': None}
Execution time: 1.253572 seconds
CPU times: user 4.08 ms, sys: 0 ns, total: 4.08 ms
Wall time: 1.25 s


## function call

参考 [LLM代码实现-Qwen（Function Calling）](https://juejin.cn/post/7330512499173507113)

In [7]:
import openai
import json
import time

openai.api_base = "http://192.168.0.50:8000/v1"
openai.api_key = "none"


def contral(response):
    if response.choices[0].message.function_call:
        result = eval(response.choices[0].message.function_call.arguments)
        switch = "开灯" if result["switch"] else "关灯"

        print(switch)


def call_qwen(messages, functions=None):
    start_time = time.time()
    print(messages)
    if functions:
        response = openai.ChatCompletion.create(
            model="Qwen", messages=messages, functions=functions
        )
    else:
        response = openai.ChatCompletion.create(model="Qwen", messages=messages)

    print(response)
    result = response.choices[0].message.function_call
    if result:
        if result.name == "contral":
            contral(response)

    print(response.choices[0].message.content)

    execution_time = time.time() - start_time
    print(f"Execution time: {execution_time:.6f} seconds")
    
    return response


if __name__ == "__main__":
    functions = [
        {
            "name_for_human": "灯光控制",
            "name_for_model": "contral",
            "description_for_model": "灯光控制可以帮助用户开关灯"
                                     + " Format the arguments as a JSON object.",
            "parameters": [
                {
                    "name": "switch",
                    "description": "用于控制灯的开关，若为True则为开灯，若为false则为关灯",
                    "required": True,
                    "schema": {"type": "bool"},
                }
            ],
        }
    ]

    
    call_qwen([{"role": "user", "content": "你是谁"}], functions)
    call_qwen([{"role": "user", "content": "帮我开一下灯"}], functions)
    call_qwen([{"role": "user", "content": "二十四节气的主要意义是啥"}], functions)
    



[{'role': 'user', 'content': '你是谁'}]
{
  "model": "Qwen",
  "object": "chat.completion",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u6211\u662f\u901a\u4e49\u5343\u95ee\uff0c\u7531\u963f\u91cc\u4e91\u5f00\u53d1\u7684\u4eba\u5de5\u667a\u80fd\u52a9\u624b\u3002\u6211\u53ef\u4ee5\u56de\u7b54\u5404\u79cd\u95ee\u9898\u3001\u63d0\u4f9b\u4fe1\u606f\u548c\u4e0e\u7528\u6237\u8fdb\u884c\u5bf9\u8bdd\u3002\u6709\u4ec0\u4e48\u6211\u53ef\u4ee5\u5e2e\u52a9\u4f60\u7684\u5417\uff1f",
        "function_call": null
      },
      "finish_reason": "stop"
    }
  ],
  "created": 1710720455
}
我是通义千问，由阿里云开发的人工智能助手。我可以回答各种问题、提供信息和与用户进行对话。有什么我可以帮助你的吗？
Execution time: 3.843729 seconds
[{'role': 'user', 'content': '帮我开一下灯'}]
{
  "model": "Qwen",
  "object": "chat.completion",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u9700\u8981\u8c03\u7528\u706f\u5149\u63a7\u5236API\u8fdb\u884c\u64cd\